<a href="https://colab.research.google.com/github/AiDAPT-A/2024-Q3-ai-in-architecture/blob/main/tutorials/T8_Similarity_Urban_Scale.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Building similarity at the urban scale

## 📌 Overview
By following this tutorial, you will learn how to conduct urban data-driven analysis at scale by computing *combined building similarity scores*.
More specifically, you will automatically extract both aerial and street view imagery from building footprints and corresponding geographical information. Then, you will integrate meaningful building image features computed via pre-trained computer vision foundation models. Finally, you will be able to visualize and interpret urban similarity maps.

<center>
<img src="https://drive.google.com/uc?export=view&id=1pWpCu7CwTzQIygkq5hHw0nLZDZLcz2Td" alt="floor-layout" class="center" width="1000px">
</center>

### 🧠 **Learning objectives**
- Visualize and interpret *urban* representations at scale
- Create a customized dataset of aerial and **street view** images
- Generate an urban similarity score by combining similarity scores computed from both aerial and street view images

### 🐍 **New in Python**
- (Reminder) HTTP requests: `requests`
- (Tips) Coordinate reference system operations: `pyproj`

### 🌍 1. Building footprint
- **Geopandas and 3D BAG**: Recall what you learned in Tutorial 4 (from building footprints to photos).

### 📸 2. Aerial and street view images
- **Boundary definition and image extraction**: Recall what you learned in Tutorials 4 and 5 (from photos to embeddings).

### 👥 3. Building similarity
- **Similarity measures**: Recall what you learned in Tutorial 5.
- **Urban similarity visualization**: Interpret and visualize building similarity at scale on a map.

### 💻 Group assignment
- **Practical exercise**: Apply what you have learned and create your own urban similarity map.

## 🏢 Building footprints: GeoPandas (3D BAG)

You very well know how to manage directories at this point. Am I right?

In [ ]:
# import drive and os libraries
from google.colab import drive
import os

# mount google drive
drive.mount('/content/drive', force_remount=True)

# set a directory of your choice
working_path = "/content/drive/MyDrive/voorhof_class"
# chdir to change the directory
os.chdir(working_path)

Let's start from the tile of building footprints collected in [Tutorial 4](https://colab.research.google.com/drive/1Ja8qsEVmx7tgDciPcKnqiOsPaPYZ3hft?usp=sharing).

In [ ]:
import geopandas
## Load GeoPandas file
path_to_data = "filtered_buildings.gpkg" # This should contain the name of the downloaded geopandas file
filtered_buildings = geopandas.read_file(path_to_data)
filtered_buildings.head()

In [ ]:
subset_buildings = filtered_buildings.iloc[0:50]  # The same 50 buildings

subset_buildings

Cross-check the file has been loaded correctly by plotting the tile.

In [ ]:
# Plot buildings footprint
import matplotlib.pyplot as plt
plt.figure(figsize=(10, 6))
subset_buildings.plot(column='b3_dak_type', legend=True, alpha=0.8) # Footprint roof type

plt.show()

### Processing geographical coordinates for each building

This time, you will extract both street view images. Following the process described in Tutorial 4, let's first find out the geographical coordinates needed to automate the image extraction process.

- **Street view images** can be retrieved via Cyclomedia's API based on a specified location in 2D geographical coordinates. The API will then send the closest street view image from the specified location. A possible choice is defining the building coordinate as the centroid of the rectangular bounding box. You can explore other options reported in the API's [documentation](https://developer.cyclomedia.com/our-apis/panorama-rendering-api/).

- (Reminder) **Aerial images** can be extracted via PDOK by defining the two corners of a rectangular bounding box. If you need more details to complete this step, do you know where to look at? Sure, you know! Tutorial 4.

<center>
<img src="https://drive.google.com/uc?export=view&id=1VjAOuUjoiafMd-TMCazMIR1cndHfBMxK" alt="floor-layout" class="center" width="500px">
</center>

In this tutorial, we will take 50 buildings (the same buildings selected in Tutorials 4 and 5), and will determine the coordinates for extracting street view images.

For bookkeeping purposes, you can append the coordinates in a dictionary and store them as json files: `build_dic_streetview.json`. \\
⚠ This will be really handy when you are working on your final project.

In [ ]:
# Load Python modules
import numpy as np
import json
from shapely.geometry import MultiPolygon
'''
get coordinates for each building and stored them in a dictionary
    - Loop over BAG IDs
    - Extract coordinates
    - Compute bounding box
    - Store results
'''

additional_padding = 10 # let's add some context (note that this variable is defined in meters)

'''
Create a dictionary build_coord where:
- Key: building_index (ind_build) # Think of this like a building ID
- Value: A list with a box coordinates

'''
build_coord = {}
street_view_coord = {}

for index, row in subset_buildings.iterrows():

    # Let's only collect the ID
    pand_id = row["identificatie"][14:]

    geometry = row.geometry  # Retrieve geometry (could be Polygon or MultiPolygon)
    if isinstance(geometry, MultiPolygon):  # Check if it's a MultiPolygon
        coords = [np.array(polygon.exterior.coords) for polygon in geometry.geoms]
    else:  # It's a single Polygon
        coords = [np.array(geometry.exterior.coords)]
    # Flatten the list if you need a single set of coordinates
    coord = np.vstack(coords)

    # Dictionary: aerial images
    coord_box = []
    coord_box.append(coord[:,0].min() - additional_padding)
    coord_box.append(coord[:,1].min() - additional_padding)
    coord_box.append(coord[:,0].max() + additional_padding)
    coord_box.append(coord[:,1].max() + additional_padding)

    # Let's make sure the bounding box is a square
    sides = np.array((coord_box[2] - coord_box[0], coord_box[3] - coord_box[1]))
    index_max = np.argmax(sides)
    index_min = np.argmin(sides)
    increment = (sides[index_max]/2) - (sides[index_min]/2)
    coord_box[index_min] = coord_box[index_min] - increment
    coord_box[index_min+2] = coord_box[index_min+2] + increment

    # Add a dictionary item
    build_coord[pand_id] = coord_box

    # Dictionary: street view images
    coord_point = []
    coord_point.append(coord[:,0].min() + (coord[:,0].max() - coord[:,0].min()) / 2)
    coord_point.append(coord[:,1].min() + (coord[:,1].max() - coord[:,1].min()) / 2)
    street_view_coord[pand_id] = coord_point

# with open("build_dic_aerial.json", 'w') as file_id:
#     json.dump(build_coord, file_id)

with open("build_dic_streetview.json", 'w') as file_id:
    json.dump(street_view_coord, file_id)

Shall we verify the generated/stored dictionary makes sense?

In [ ]:
street_view_coord

## 📸 (Recap) Retrieving street view images via Cyclomedia's API

You already know how to use Cyclomedia's API. The focus here will hence be on automatically retrieving street view images.

(Bookkeeping) Where we are at?

In [ ]:
os.getcwd()

Below, we will load the required Python module to launch URL requests and create a function to ask Cyclomedia's API the street view images you would like to extract.

In [ ]:
# As usual in Python, let's load the necessary modules
import requests
from requests.auth import HTTPBasicAuth

The get_image function allows you to automatically download a panoramic image from Cyclomedia's API by specifying a location and how you want to view it.

What the function does:
+ Takes input coordinates (COOR_X and COOR_Y) in a local coordinate system (CRS 28992).

+ Constructs a URL to request a panorama image from Cyclomedia using those coordinates.

+ Sends a request to the Cyclomedia server, asking for an image with specific view settings (like field of view, direction, and image size).

+ Saves the image locally with a name you choose.

+ Returns the image path if successful, or prints an error if something goes wrong.

Important inputs you can adjust:
+ `name_image`: what you want to call the image file.

+ `path_image`: folder or location where you want to save the image.

+ `hfov`: horizontal field of view (in degrees) — wider values show more of the scene.

+ `pitch`: up/down angle of the camera.

+ `yaw`: left/right angle (direction you're looking).

+ `width and height`: size of the image in pixels.

In [ ]:
def get_image(COOR_X,
            COOR_Y,
            name_image='image_0',
            path_image='',
            hfov='90',
            pitch='0',
            yaw='0',
            width='1200',
            height='1200',
            index='0'):

    # ! Specify you user and pass here (very private) :)
    username = ' ' #Your user here
    password = ' ' #Your pass here

    # Construct the URL to connect with the API
    URL_START = 'https://atlasapi.cyclomedia.com/api/PanoramaRendering/RenderByLocation2D/'
    CRS = '28992'
    API_KEY = '2XU9ajAxS3YtKvgVCD4h-30lK3IpOb8x-HApGTcRCf09a7UVuX8h-r4ZarYZ2nIB'
    # COOR_X = '94313.15'
    # COOR_Y = '435711.11'
    HFOV = hfov
    PITCH = pitch
    YAW = yaw
    WIDTH = width
    HEIGHT = height
    INDEX = index

    file_name = path_image + name_image + '.jpg'

    url = URL_START \
    + CRS + '/' \
    + COOR_X + '/' + COOR_Y \
    + '/?width=' + WIDTH + '&height=' + HEIGHT \
    + '&hfov=' + HFOV \
    + '&apiKey=' + API_KEY \
    + '&pitch=' + PITCH + '&yaw=' + YAW \
    + '&index=' + INDEX

    # Contact the API and request the images
    response = requests.get(url, auth=HTTPBasicAuth(username, password))

    if response.status_code == 200: # This means everything went well
        with open(file_name, 'wb') as f:
            f.write(response.content) # Store the images
        return file_name
    else: # If there was an error, we need to know about it!
        print('Error: ', response.status_code)
        return None

Where would you like to store all the street view images? Define the directory below. Note that the folder should exist in your GDrive.

In [ ]:
#directory where the street view images will be stored
street_view_images_path = "/content/drive/MyDrive/voorhof_class/street_view_images/"

Just a reminder: let's check how a single image can be retrieved and we will then automate the process.

In [ ]:
# Only one image (easy!)
# coor_x = '68554.7'
# coor_y = '445294.4'
# get_image(coor_x, coor_y, 'image_0', street_view_images_path, yaw='0')

Well, you have work very hard up to this point. It is time for your PC (or should we say Collab?) to do the work for you by automating the process.

Iterate over the geographical coordinates determined for each building footprint and request the corresponding street view image. Unless Cyclomedia already limited your quota, this should work! Enjoy seeing how street images are flowing to your local directory. 😎

In [ ]:
for build_id, coord in street_view_coord.items():
  get_image(str(coord[0]), str(coord[1]), str(build_id), street_view_images_path, yaw='0')
  #print(build_id, coord)

## 💻 Computing **urban** embeddings via DinoV2

Being that we are ambitious, we will not stop at computing embeddings (meaningful representations) separetely for aerial and street view images, we will compute similarity scores for both!

<center>
<img src="https://drive.google.com/uc?export=view&id=1hXzNURGqRK8Si30BJoDICV8RDtTOintb" alt="floor-layout" class="center" width="400px">
</center>

To speak Dino's language, transform the extracted images as shown below and stored the transformed street view images as a torch tensor.
Let's also load the previously computed *aerial image* embeddings. Certainly, you are familiar with this process!

In [ ]:
# Get vector embeddings from a aerial images
# Load modules
import torch
import os
from PIL import Image
import torchvision.transforms as T

In [ ]:
aerial_embed = torch.load('embeddings/vorhof_0_50.pt') # Load previously computed embeddings (from building aerial images)

In [ ]:
print(aerial_embed.shape) # Shape... check!

In [ ]:
# This is the directory where the extracted images are stored:
# street_view_images_path #

# Specify here which transformations will be applied to the image
transform = T.Compose([
    T.Resize(224, interpolation=T.InterpolationMode.BICUBIC),
    T.ToTensor(),
    T.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
])

# Prepare a list with all images filenames to run everything in a loop fashion
transformed_images = [] # Initialize a list for the transformed images
list_build_images = [] # This list will contain the building ids

# And here comes the loop iterating over all images
for img_filename in os.listdir(street_view_images_path):
    img_path = os.path.join(street_view_images_path, img_filename) # Combine path and image name
    list_build_images.append(img_filename.split('.')[0]) # Append the building id to the list
    print(img_path) # Let's print out the processed images

    if os.path.isfile(img_path):
        img = Image.open(img_path) # The image is opened by PIL Image function (Think of PIL as a translator)
        t_img = transform(img) # The transformation is now applied here
        transformed_images.append(t_img)  # Append the transformed image to the list

# Convert list of tensors to a single 4D tensor (Recall: one image would be a 3D tensor)
tensor_streetview_images = torch.stack(transformed_images)

In [ ]:
# tensor_images is now a 4D tensor of shape [N, C, H, W], where N is the number of images
print(tensor_streetview_images.shape)

Let's summon Dino up to our collab notebook and compute embeddings from our street view images.

In [ ]:
dinov2_vits14_reg = torch.hub.load('facebookresearch/dinov2', 'dinov2_vits14_reg')

In [ ]:
# Let's compute embeddings from street view imagery
with torch.no_grad():
    streetview_embed = dinov2_vits14_reg(tensor_streetview_images)

print(aerial_embed.shape, streetview_embed.shape) # Just to check the shape of the tensor

## 👁 Urban similarity based on cosine similarity

Great, you can now rely on the embeddings computed from both aerial and street-view images and perform a variety of tasks.

In this tutorial, we will again focus on information retrieval and will determine (dis)similar buildings based on urban cosine similarity. To do so, we will combine the cosine similarity scores calculated for both aerial and street-view images. You can decide which importance weight should be assign to each of them.

Reusing a function defined in Tutorial 5, we can compute the cosine similarity scores among all picked buildings.

In [ ]:
# Define the function to compute cosine similarity scores
def cosine_similarity(x):
    num_samples = x.size(0) # Size of our tensor
    similarity = torch.zeros((num_samples, num_samples)) # Initialize a tensor where the scores will be stored

    # Looping over the elements in the tensor
    # We will compute the similarity scores of all elements with all elements
    for i in range(num_samples):
        for j in range(num_samples):
            # Compute the similarity score between x[i] and x[j]
            similarity[i, j] = torch.dot(x[i], x[j]) / torch.norm(x[i], p=2) / torch.norm(x[j], p=2)

    return similarity

In fact, let's calculate cosine similarity scores based on the embeddings computed for all cases: (i) aerial images, (ii) street view images, (iii) combined urban similarity scores.

In [ ]:
similarity_aerial = cosine_similarity(aerial_embed)
similarity_streetview = cosine_similarity(streetview_embed)

We are combining two types of visual similarity to compute an overall urban similarity score:

- `similarity_aerial`: measures how similar two locations are based on **aerial imagery** (top-down view).
- `similarity_streetview`: measures similarity using **street-level images** (what you see at eye level).

The weights define how much each view contributes to the final score:

```python
w_aerial = 0.5
w_streetview = 0.5
```
In this example, both views are given equal importance. Feel free to adjust the weights when you calculate urban similarity scores for your final project.

In [ ]:
w_aerial = 0.5
w_streetview = 0.5
similarity_urban = w_aerial * similarity_aerial + w_streetview * similarity_streetview

In [ ]:
similarity_urban.shape

Not strictly necessary, but you can visualize the cosine similarity scores for all examined cases.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from mpl_toolkits.axes_grid1 import make_axes_locatable

fig, axs = plt.subplots(1, 3, figsize=(15, 5))

titles = ["Similarity: Aerial", "Similarity: Street", "Similarity: Urban"]
cmaps = ['Blues', 'Reds', 'Greens']
datas = [similarity_aerial, similarity_streetview, similarity_urban]


for ax, title, cmap, d in zip(axs, titles, cmaps, datas):
    im = ax.imshow(d, cmap=cmap, vmin=0, vmax=1)
    ax.set_title(title)
    ax.set_xlabel("Building Index")
    ax.set_ylabel("Building Index")

    # Attach a colorbar of matching height
    divider = make_axes_locatable(ax)
    cax = divider.append_axes("right", size="5%", pad=0.05)
    plt.colorbar(im, cax=cax)

plt.tight_layout()
plt.show()

You will now select one building (out of the 50) and will explore similar buildings based on the computed similarity scores.

In [ ]:
# Similarity with respect to one building (index = 0)
similarity_query_aerial = similarity_aerial[0].tolist() # Note torch tensors are converted to lists here
similarity_query_streetview = similarity_streetview[0].tolist()
similarity_query_urban = similarity_urban[0].tolist()

## 🗺 Visualizing building similarity

You have now computed the similarity score between a selected (query) building and a gallery of 50 buildings. Wouldn't it be great to visualize the results on a map? Let's do it. Are you more comfortable coding Python or interacting with QGIS interface? Either way will work!

To visualize building similarity on a map, you will attach the computed similarity scores as additional columns to the original geopandas tile. Yes, the cycle is closing and we are moving back to the tile.


In [ ]:
id_to_index = {key: i for i, key in enumerate(street_view_coord.keys())}

In [ ]:
id_to_index

In [ ]:
# First, we add (sort of empty) columns to the tile. One for each case.
subset_buildings = subset_buildings.copy()
subset_buildings['aerial_similarity'] = np.nan
subset_buildings['streetview_similarity'] = np.nan
subset_buildings['urban_similarity'] = np.nan

# Create a mapping from pand_id to index for faster lookup
id_to_index = {key: i for i, key in enumerate(street_view_coord.keys())}

# Then, we iterate over all 50 building footprints and include the corresponding similarity.
for idx, row in subset_buildings.iterrows():

    # Let's only collect the ID
    pand_id = row["identificatie"][14:]

    if pand_id in id_to_index:
        i = id_to_index[pand_id]

        subset_buildings.loc[idx, 'aerial_similarity'] = similarity_query_aerial[i]
        subset_buildings.loc[idx, 'streetview_similarity'] = similarity_query_streetview[i]
        subset_buildings.loc[idx, 'urban_similarity'] = similarity_query_urban[i]

Let's now store our modified geopandas file, which contains now building similarity information.

In [ ]:
# Store GeoPandas
subset_buildings.to_file("urban_similarity.gpkg", driver='GPKG')

### Visualization directly on Python

You can now visualize, set up, and store plots directly on Python. Alternatively, this task can be done via QGIS' interface.

Let's plot, for instance, urban similarity. Note that you can refine the code to produce a figure that satisfy your needs.

In [ ]:
subset_buildings.plot(column='urban_similarity', legend=True, alpha=0.8, cmap='RdYlBu')

Another example below. Here building similarity is visualized based on similarity scores computed from aerial, street view images, and a combination thereof.

In [ ]:
# Create a figure and 3 subplots, organized horizontally (1 row, 3 columns)
fig, axs = plt.subplots(1, 3, figsize=(15, 5))

# Plotting the same 'column' with different color maps on each subplot
subset_buildings.plot(column='aerial_similarity', ax=axs[0], legend=True, alpha=0.8, cmap='coolwarm_r', vmin=0, vmax=1)
subset_buildings.plot(column='streetview_similarity', ax=axs[1], legend=True, alpha=0.8, cmap='coolwarm_r', vmin=0, vmax=1)
subset_buildings.plot(column='urban_similarity', ax=axs[2], legend=True, alpha=0.8, cmap='coolwarm_r', vmin=0, vmax=1)

# Set titles for each subplot for clarity
axs[0].set_title('Aerial similarity')
axs[1].set_title('Street view similarity')
axs[2].set_title('Urban similarity')

# Remove axis labels and tick labels for all subplots
for ax in axs:
    ax.set_xlabel('')
    ax.set_ylabel('')
    ax.set_xticklabels([])
    ax.set_yticklabels([])

# Adjust layout to make room for the legend
plt.tight_layout()

# Display the plot
plt.show()

(Optional) You can also visualize the images corresponding to certain building by following the code below. I know you are curious enough and would like to verify the results.


In [ ]:
query_index = "0503100000000172"
building_index = "0503100000038135"

# Create the matplotlib grid for visualization
fig, axs = plt.subplots(2, 2, figsize=(2 * 4, 1 * 4))

aerial_image_ = "aerial_images/"+ query_index +".jpg"
image_sim = np.array(Image.open(aerial_image_))
axs[0, 0].imshow(image_sim)
axs[0, 0].set_title("query: aerial")  # Optional: Show filename as title
axs[0, 0].axis('off')  # Hide axes for better visualization

aerial_image_ = "aerial_images/" + building_index +".jpg"
image_dis = np.array(Image.open(aerial_image_))
axs[0, 1].imshow(image_dis)
axs[0, 1].set_title("building: aerial")  # Optional: Show filename as title
axs[0, 1].axis('off')  # Hide axes for better visualization

streetview_image_ = "street_view_images/"+ query_index +".jpg"
image_sim = np.array(Image.open(streetview_image_))
axs[1, 0].imshow(image_sim)
axs[1, 0].set_title("query: street")  # Optional: Show filename as title
axs[1, 0].axis('off')  # Hide axes for better visualization

streetview_image_ = "street_view_images/"+ building_index +".jpg"
image_dis = np.array(Image.open(streetview_image_))
axs[1, 1].imshow(image_dis)
axs[1, 1].set_title("building: street")  # Optional: Show filename as title
axs[1, 1].axis('off')  # Hide axes for better visualization

plt.tight_layout()
plt.show()

(Optional) And finally, let's set up our **building search engine**:

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image

def building_search(building_id, id_to_index, similarity_urban, subset_buildings, number_similar_buildings=3):
    """
    - building_id: the query building ID (string).
    - id_to_index: dict mapping building IDs to indices (for accessing similarity_urban).
    - similarity_urban: a tensor/array of similarity scores, where similarity_urban[idx_list]
      gives similarity scores to all other buildings from the query building index.
    - subset_buildings: GeoDataFrame containing building footprints (with column 'identificatie').
    - number_similar_buildings: how many similar buildings to retrieve and plot.

    This function returns the IDs of the query building + similar buildings and
    also shows:
      1) a plot comparing their aerial and street view images, and
      2) a plot of building footprints colored by similarity to the query building.
    """
    # -- 1) Identify top-k similar buildings --
    idx_list = id_to_index[building_id]  # retrieve index based on query building's ID

    # similarity vector with respect to the query building
    similarity_urban_ = similarity_urban[idx_list]

    # topk returns (values, indices); we get the top-k + the query building itself
    similarity_ranking, idx_ranking = similarity_urban_.topk(number_similar_buildings + 1)

    # match these indices back to building IDs
    all_building_ids = list(id_to_index.keys())
    building_ids_ranking = [all_building_ids[i] for i in idx_ranking.tolist()]

    # Building IDs: first one is the query building, subsequent ones are similar buildings
    print("Query building: ", building_ids_ranking[0])
    print("Similar buildings: ", building_ids_ranking[1:])

    # -- 2) Plot the query and top-k similar buildings --
    # We'll make a figure with 2 rows and (k + 1) columns
    fig, axs = plt.subplots(
        2,                       # two rows (row 0 = aerial, row 1 = street view)
        number_similar_buildings + 1,  # columns = query + similar
        figsize=((number_similar_buildings + 1) * 4, 2 * 4)  # adjust figure size
    )

    for col, b_id in enumerate(building_ids_ranking):
        # Aerial image
        aerial_path = f"aerial_images/{b_id}.jpg"
        try:
            aerial_img = np.array(Image.open(aerial_path))
            axs[0, col].imshow(aerial_img)
        except FileNotFoundError:
            axs[0, col].text(0.5, 0.5, 'No aerial image found', ha='center', va='center')

        axs[0, col].axis('off')
        if col == 0:
            axs[0, col].set_title(f"Query aerial: {b_id}")
        else:
            axs[0, col].set_title(f"Sim. {col} aerial: {b_id}")

        # Street view
        street_path = f"street_view_images/{b_id}.jpg"
        try:
            street_img = np.array(Image.open(street_path))
            axs[1, col].imshow(street_img)
        except FileNotFoundError:
            axs[1, col].text(0.5, 0.5, 'No street image found', ha='center', va='center')

        axs[1, col].axis('off')
        if col == 0:
            axs[1, col].set_title(f"Query street: {b_id}")
        else:
            axs[1, col].set_title(f"Sim. {col} street: {b_id}")

    plt.tight_layout()
    plt.show()

    # -- 3) Add similarity columns to a copy of your building footprints GeoDataFrame --
    subset_build = subset_buildings.copy()
    subset_build['aerial_similarity'] = np.nan
    subset_build['streetview_similarity'] = np.nan
    subset_build['urban_similarity'] = np.nan

    # For each row in the GeoDataFrame, fill in the similarity value for that building
    for idx, row in subset_build.iterrows():
        # Extract building ID (e.g., skipping first 14 chars if that's correct for your data)
        pand_id = row["identificatie"][14:]

        if pand_id in id_to_index:
            i = id_to_index[pand_id]
            # similarity_urban_ is the entire similarity vector for the query building vs. all others
            # so the similarity for building i is:
            subset_build.loc[idx, 'urban_similarity'] = similarity_urban_[i].tolist()
            # If you had separate arrays for aerial/street similarity, you’d do:
            # subset_build.loc[idx, 'aerial_similarity'] = similarity_aerial_[i]
            # subset_build.loc[idx, 'streetview_similarity'] = similarity_street_[i]

    # -- 4) Plot the GeoDataFrame colored by similarity (e.g., 'urban_similarity') --
    # Example: 1 row, 1 subplot for just 'urban_similarity'.
    # If you also want to show 'aerial_similarity' and 'streetview_similarity',
    # you can make multiple subplots. Below is a single-subplot example.


    # ---------------------------------------------------------
    # -- 4) Plot the GeoDataFrame colored by 'urban_similarity'
    #      and highlight the query + 3 similar buildings
    # ---------------------------------------------------------

    # 1) Separate out query vs. similar IDs
    query_id = building_ids_ranking[0]      # first is the query
    similar_ids = building_ids_ranking[1:]  # the next 'number_similar_buildings' are similar

    # 2) Make small GeoDataFrames for the query and similar sets
    query_building = subset_build[ subset_build["identificatie"].str[14:] == query_id ]
    similar_buildings = subset_build[ subset_build["identificatie"].str[14:].isin(similar_ids) ]

    # Base map (polygons colored by similarity)
    fig, ax = plt.subplots(1, 1, figsize=(15, 6))
    subset_build.plot(
        column='urban_similarity',
        ax=ax,
        legend=True,           # colorbar for the similarity column
        alpha=0.8,
        cmap='coolwarm_r',
        vmin=0,
        vmax=1,
        label='_nolegend_'     # don't put these polygons in the legend
    )

    # We only want a "dot" marker for query/similar buildings.
    # Polygons can't be displayed with a marker symbol, so we convert them to point centroids:
    query_centroid = query_building.copy()
    query_centroid["geometry"] = query_centroid.geometry.centroid

    similar_centroids = similar_buildings.copy()
    similar_centroids["geometry"] = similar_centroids.geometry.centroid

    # Plot the query centroid as a red 'X'
    query_centroid.plot(
        ax=ax,
        marker='X',
        color='red',
        markersize=4,
        label='Query Building'
    )

    # Plot the similar buildings as black circles
    similar_centroids.plot(
        ax=ax,
        marker='o',
        color='black',
        markersize=4,
        label='Similar Buildings'
    )

    # Final formatting
    ax.set_title('Urban similarity')
    ax.set_xlabel('')
    ax.set_ylabel('')
    ax.set_xticklabels([])
    ax.set_yticklabels([])

    # This automatically handles the two point layers (query & similar) in the legend
    ax.legend(loc='upper right')

    plt.tight_layout()
    plt.show()

    return

In [ ]:
# 0503100000037163 (standard building)
# 0503100000000172 (large building)

building_search("0503100000037163", id_to_index, similarity_urban, subset_buildings, number_similar_buildings=3)

## 🧙 Additional hints and tricks

**Merging geopandas files**
Say you would like to combine two (or more) tiles of building footprints (maybe) retrieved from 3D BAG. You just have to concatanate the individual geopandas as shown below.

In [ ]:
import pandas as pd

# Load another tile
path_to_data = "second_tile.gpkg"
second_tile = geopandas.read_file(path_to_data)
# Concatenate geopandas
tile_concat = pd.concat([tile, second_tile], ignore_index=True)

In [ ]:
# Plot buildings footprint
import matplotlib.pyplot as plt
plt.figure(figsize=(10, 10))
tile_concat.plot(column='b3_dak_type', legend=False, alpha=0.8) # Footprint roof type
plt.show()

**Transform geographical coordinates**

In [ ]:
from pyproj import Transformer

# Create a transformer object for converting from EPSG:4326 to EPSG:28992
transformer = Transformer.from_crs("EPSG:4326", "EPSG:28992", always_xy=True)

# Example coordinates from Google Maps (longitude, latitude)
google_maps_coordinate = (4.475639, 51.922667)  # Longitude, Latitude

# Transform the coordinate
rd_new_coordinate = transformer.transform(*google_maps_coordinate)

print(f"RD New Coordinate: {rd_new_coordinate}")

**(Bonus info)**
You can find the geographical coordinates related to all buildings investigated in the group projects.

- Lumiere: 51°55'21.6"N 4°28'32.3"E. // 51.922667, 4.475639.
- Strijp-S: 51°26'48.8"N 5°27'31.9"E. // 51.446889, 5.458861.
- The Stack: 52°23'14.7"N 4°54'13.4"E. // 52.387417, 4.903722.
- SPOT: 52°18'25.6"N 4°56'45.1"E. // 52.307111, 4.945861.

### Visualization via QGIS interface

**Import GeoPandas file**

To import a GeoPandas package, go to the data source manager and upload the file as indicated in the illustration below.

<center>
<img src="https://drive.google.com/uc?export=view&id=1YHWE6roq3vuKu0V6gFnhJKvZqv5sa2JE" alt="floor-layout" class="center" width="500px">
</center>


**Set up visualization settings**

GeoPandas loaded, yeah! You can now color-code building footprints according to the computed similarity scores by changing the properties of the layer. Specifically, you can color-code items categorically or gradually. The latter can be very useful to visualize similarity measures. But, of course, you are free to decide the final setup.

<center>
<img src="https://drive.google.com/uc?export=view&id=1Xeh6hTlppwydBgIGDY819TEJSZjwZD0O" alt="floor-layout" class="center" width="500px">
</center>

**Export canvas (map)**

Once you have finalized adjusting the colormap (surely you will take some time to make up your mind), you can then export the map as indicated below. There are two options:
- Fast export: you can quickly export the map as a pdf/image.
- Detailed export: additional settings can be carefully selected if you create a new print layout.

You can find more info [here](https://docs.qgis.org/3.34/en/docs/training_manual/map_composer/map_composer.html).

<center>
<img src="https://drive.google.com/uc?export=view&id=1CGSxUEUTfdHf3Vg2dBmKV2yIHwG9-39v" alt="floor-layout" class="center" width="500px">
</center>

# ✅ Assignment: Represent your own urban similarity map!

- 1p: retrieve street view images for each building footprint contained in the tile corresponding to your project. Retrieve images for the same buildings assigned to you in Tutorial 4.
- 1p: compute the embedding representations of the collected street view images through DINOv2, store them, and share your local directory (see more instructions below).
- 2p: extract the footprint for a **query building** different from the buildings in your gallery. It could be your project's building or other building of your choice. Tip: you may postprocess your query aerial and/or street view images so that they resemble most of the to-be-built building if the project has not been completed yet.
- 1p: collect the aerial and street-view images corresponding to your query building
- 1p: compute the embedding for your query building via DINOv2
- 2p: calculate the urban similarity between your query building and all buildings contained in your local gallery
- 2p: visualize the computed urban similarity on a map, color-coding buildings according to their similarity score to your *query* building.

### **Output**</br>
**Write your findings and interpretation in a notebook** named **"A7_urban_similarity_\<name\>.ipynb"**.

**Share the folder** containing the collected building street-view images.

! Please include the links pointing to the notebook and local folder when submitting the assignment on Brightspace.

- Deadline: Friday, April 11

⏰ Street-view images must by shared by **Monday, April 7**. You can share the link (per person or group) to `p.g.moratodominguez@tudelft.nl`.

**(Important)** The images must be named according to the building's PAND ID. Example: `0503100000037135.jpg`.


## Additional guidelines

#### Neighborhood
You will collect building images from a specific neeighborhood within the `Hof van Delft` district, in Delft. Each group will deal with a specific neighborhood:

- Group 1: `Bedrijventerrein Altena`
- Group 2: `Agnetaparkbuurt`
- Group 3: `Ministersbuurt-West`
- Group 4: `Ministersbuurt-Oost`
- Group 5: `Westeindebuurt`
- Group 6: `Olofsbuurt`
- Group 7: `Krakeelpolder`
- Group 8: `Westerkwartier`

#### Subset
Coordinate with your group to decide the subset of buildings you will individually focus on.

#### Data storage
Place the street view images and dictionary into a local folder.

👉 You can download building and neighborhood data for the `Hof van Delft` district [here](https://drive.google.com/drive/folders/1r6RPCXV_DLK2EdQHnqhCL2oimnBZL_hM?usp=sharing)